# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.decomposition import PCA

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization

from sklearn.model_selection import train_test_split
import keras.backend as K
from scipy.stats import pearsonr
from sklearn.metrics import accuracy_score, precision_score, recall_score
## required for effictrain_test_splitnt GPU use
import tensorflow as tf
from keras.backend import tensorflow_backend
from sklearn.manifold import TSNE
from datetime import datetime
from matplotlib import pyplot as plt
from keras import backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, Dropout, Activation, Flatten, MaxPooling2D

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
# config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
# session = tf.Session(config=config)
# tensorflow_backend.set_session(session)
## required for efficient GPU use


Using TensorFlow backend.


In [2]:
def precision(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
   
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
   
    return fbeta_score(y_true, y_pred, beta=1)

# Read Dataset

In [3]:
path = os.getcwd()
path = os.path.join(path, "Dataset")
subdistrict_income = pd.read_csv(path + "/income.csv")
regional_info = pd.read_csv(path + "/region.csv")
features_info = pd.read_csv(path + "/devlopmental_data.csv")

In [4]:
# columns = features_info.columns
# print(columns)
# columns = columns[:-2]
# ls = []
# for c in columns:
#     ls.append(int(c))
# ls = sorted(ls)
# features = pd.DataFrame()
# for l in ls:
#     features    = pd.concat([features, features_info[str(l)]], axis=1, ignore_index=True).reset_index(drop=True)
# features = pd.concat([features, features_info['Town/Village_code'], features_info['Subdistt_code']], axis=1, ignore_index=True).reset_index(drop=True)

In [5]:
# features[26] = features[26].astype(np.int64)

In [6]:
# features = features[features[26].isin(subdistrict_income['Subdistt_code'].values)].reset_index(drop=True)

In [7]:
feature = features_info.copy(deep=True)
cols = feature.columns
cols = cols[3:]
counter = 0
for i in range(feature.shape[0]):
    if counter %1000 == 0:
        print("Counter : ",counter)
    counter = counter +1
    num_hh = feature.iloc[i]['Household']
    feature.at[i, cols] = feature.iloc[i][cols]*num_hh
  

Counter :  0
Counter :  1000
Counter :  2000
Counter :  3000
Counter :  4000
Counter :  5000
Counter :  6000
Counter :  7000
Counter :  8000
Counter :  9000
Counter :  10000
Counter :  11000
Counter :  12000
Counter :  13000
Counter :  14000
Counter :  15000
Counter :  16000
Counter :  17000
Counter :  18000
Counter :  19000
Counter :  20000
Counter :  21000
Counter :  22000
Counter :  23000
Counter :  24000
Counter :  25000
Counter :  26000
Counter :  27000
Counter :  28000
Counter :  29000
Counter :  30000


In [8]:
aggregated_subdistrict = feature.groupby("Subdistt_code").sum()
print(aggregated_subdistrict)
inds = aggregated_subdistrict.index
print(inds)
for i in inds:
    aggregated_subdistrict.loc[i, cols]  /= aggregated_subdistrict.loc[i]['Household']

               Town/Village_code  Household       0_1     0_2        0_3  \
Subdistt_code                                                              
701                     21592453      81646  11124.45   80.19    415.733   
702                     28884349     104510  18597.60   39.66    547.292   
703                     27901335      88747  30525.20   64.60   2364.573   
704                     17293265      67992  18674.90   28.90    340.218   
717                     22699131      72700  20568.55   53.54    510.287   
...                          ...        ...       ...     ...        ...   
1005                    74585732     114590  20495.12  259.03  77756.970   
1006                    45203524      82040  27944.31  527.08  43899.940   
1007                    39684537      65200  16041.97   93.42    274.195   
1008                    78639660     121310  16306.86   32.04    585.760   
1009                    28836792      48262   8027.91   53.71    103.303   

           

In [9]:
y = subdistrict_income.sort_values(['Subdistt_code']).reset_index(drop=True)

In [10]:
y = y[['00','01', '02']]

In [11]:
subdistrict_income = subdistrict_income.drop_duplicates(subset=None, keep='first', inplace=False)

In [12]:
cols = feature.columns
cols = cols[3:]
train_data = aggregated_subdistrict[cols]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_data.values, y.values, test_size=0.3)

In [14]:
def pca_plot(dataset_main,label_main,title,classes):
    #TSNE Plot for glass dataset
    tsne = PCA(n_components=2)
    tsne_results = tsne.fit_transform(dataset_main)

    df_subset = pd.DataFrame()
    df_subset['X'] = tsne_results[:,0]
    df_subset['y']=label_main
    df_subset['Y'] = tsne_results[:,1]
    plt.figure(figsize=(6,4))
    plt.title(title)
    sns.scatterplot(
        x="X", y="Y",
        hue="y",
        palette=sns.color_palette("hls", classes),
        data=df_subset,
        legend="full",
        alpha=1.0
    )

# Build Model

In [21]:
def build_model2():
    
    model = Sequential()
    
    model.add(BatchNormalization(input_shape=(25,)))
    model.add(Dense(8, input_shape=(25,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(4, input_shape=(8,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(4, input_shape=(4,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(3, input_shape=(4,)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    return model

In [22]:
model = build_model2()

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

ValueError: ('Could not interpret optimizer identifier:', <keras.optimizers.RMSprop object at 0x7fb6645c1f10>)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 25)                100       
_________________________________________________________________
dense (Dense)                (None, 8)                 208       
_________________________________________________________________
batch_normalization_1 (Batch (None, 8)                 32        
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
batch_normalization_2 (Batch (None, 4)                 16        
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test),shuffle=True)

In [ ]:
y_prob = model.predict(X_test)
print("Corelation for each Class")
print("For roof classes:", pearsonr(y_test[:, 0], y_prob[:, 0]))
print("For Electricity classes:", pearsonr(y_test[:, 1], y_prob[:, 1]))
print("For water sources classes:", pearsonr(y_test[:, 2], y_prob[:, 2]))

threshold = 0.1
threshold_1= []
accuracy_score_1 = []
precision_score_1 = []
recall_score_1 = []

while threshold<1.0:
    threshold_1.append(threshold)

    p1m = np.copy(y_test[:, 0])
    p1m[p1m >= threshold] = 1
    p1m[p1m < threshold] = 0
    frac = np.sum(p1m) / len(p1m)
    ot = [1 if i >= threshold else 0 for i in y_test[:, 0]]
    pt = [1 if i >= threshold else 0 for i in y_prob[:, 0]]
    print("------------------------------------------------------------------------------------")
    print(
        "Threshold: " + str(threshold)
        + " Accuracy: " + str(accuracy_score(ot, pt))
        + " Baseline: " + str(max(frac, 1 - frac))
        + " Precision: " + str(precision_score(ot, pt))
        + " Recall: " + str(recall_score(ot, pt)))
    accuracy_score_1.append(accuracy_score(ot, pt))
    precision_score_1.append(precision_score(ot, pt))
    recall_score_1.append(recall_score(ot, pt))
    threshold = threshold +  0.1 
    
# score = model.evaluate(X_test, y_test, verbose=0)

# #print loss and accuracy
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(threshold_1,recall_score_1,'r--',label="Recall")
plt.plot(threshold_1,accuracy_score_1,'b--',label="Accuracy")
plt.plot(threshold_1,precision_score_1,'y--',label="Precision")
plt.xlabel("Thresholds")
plt.ylabel("Evaluation Metric")
plt.legend()
plt.show()
plt.close()


In [ ]:
pca_plot(X_train,p1m,"Visualisation for Census Dataset for selected Subdistricts",2)

In [ ]:
# Model 2 for feature Extraction 

In [ ]:
image_input = Input(shape=(400, 400, 3), name='image_input')

shared = Conv2D(512, (3, 3), padding='same', activation='relu')(image_input)
shared = MaxPooling2D((2, 2), padding='same')(shared)
shared = Conv2D(128, (3, 3), padding='same', activation='relu')(shared)
shared = MaxPooling2D((2, 2), padding='same')(shared)
shared = Dropout(0.25)(shared)
shared = Conv2D(32, (3, 3), padding='same', activation='relu')(shared)
shared = MaxPooling2D((2, 2), padding='same')(shared)
shared = Dropout(0.25)(shared)
shared = Conv2D(8, (3, 3), padding='same', activation='relu')(shared)
shared = MaxPooling2D((2, 2), padding='same')(shared)
shared = Dropout(0.25)(shared)
shared = Flatten()(shared)

inputA = Dense(4096, activation="relu")(shared)
inputA = Dense(2048, activation="relu")(inputA)
inputA = Dense(2048, activation="relu")(inputA)
inputA = Dense(1024, activation="relu")(inputA)
inputA = Dense(9, activation="relu")(inputA)

inputB = Dense(4096, activation="relu")(shared)
inputB = Dense(2048, activation="relu")(inputB)
inputB = Dense(2048, activation="relu")(inputB)
inputB = Dense(1024, activation="relu")(inputB)
inputB = Dense(6, activation="relu")(inputB) 

inputC = Dense(4096, activation="relu")(shared)
inputC = Dense(2048, activation="relu")(inputC)
inputC = Dense(2048, activation="relu")(inputC)
inputC = Dense(1024, activation="relu")(inputC)
inputC = Dense(10, activation="relu")(inputC)
 
model = Model(inputs=[image_input], outputs=[inputA,inputB,inputC])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, validation_data=([X_test], y_test))